# TeamBadass Milestone 2

This notebook intends to explore and describe the relevant aspects of our data sets, preprocess it, and perform some initial analysis. The first part of the notebook contains the loading of data and some general preprocessing used throughout the notebook. The rest of the notebook is divided into the five research questions specified in [**README**](https://github.com/epfl-ada/ada-2022-project-teambadass/blob/main/README.md). Each section contains specific preprocessing for the task and necessary statistics and plots. The initial analysis we have performed intends to give us a clue whether our choice of methods is reasonable and sound. 

---
**Contents of notebook:**
- [Data Processing](#Data-Processing)
  - [Loading data](#Loading-data)
  - [Removing null values](#Removing-null-values)
- [Release Date](#Release-Date)
  - [Data Processing](#Data-Processing)
  - [Plots - Year](#Plots---Year)
  - [Plots - Month](#Plots---Month)
  - [Adjusting for Inflation](#Adjusting-for-Inflation)
  - [Statistics](#Statistics)
- [Diversity](#Diversity)
  - [Distributions](#Distributions)
  - [Ethnicities](#Ethnicities)
- [Cast](#Cast)
  - [Preprocessing](#Preprocessing)
  - [Understanding Cast Data](Understanding-Cast-Data)
  - [Exploring Method: Linear Regression](#Exploring-Method:-Linear-Regression)
- [Runtime](#Runtime)
  - [Data Preprocessing](#Data-Preprocessing)
  - [Distribution of runtime](#Distribution-of-runtime)
  - [Regression & Scatterplot](#Regression-&-Scatterplot)
  - [Effect of short runtime on revenue](#Effect-of-short-runtime-on-revenue)
  - [Difference in runtime between blockbuster vs. non-blockbuster](#Difference-in-runtime-between-blockbuster-vs.-non-blockbuster)
- [Plot Summary](#Plot-Summary)
  - [Data Processing](#Data-Processing)
  - [Regression & Scatterplot](#Regression-&-Scatterplot)
  - [Difference in revenues between movies with more positive and more negative plot](#Difference-in-revenues-between-movies-with-more-positive-and-more-negative-plot)
  - [Difference in sentiment proportion between blockbuster vs. non-blockbuster](#Difference-in-sentiment-proportion-between-blockbuster-vs.-non-blockbuster)

---
## Data Processing

---
### Loading data

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import requests
from datetime import datetime as dt
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy import stats
from helpers import *

In [2]:
# turn off warning
# pd.options.mode.chained_assignment = None

In [3]:
# constants

# data folder
DATA_FOLDER = "./data/"

# google disk
MOVIE_PATH = get_path(
    "https://drive.google.com/file/d/18ZLIKZsH41qls54Gy1qSYW1xtovf9Ke9/view?usp=share_link"
)
CHARACTER_PATH = get_path(
    "https://drive.google.com/file/d/1b3_Jn3bBJl6prrtPagU-Yol-ijFMod2u/view?usp=share_link"
)
RATING_PATH = get_path(
    "https://drive.google.com/file/d/1Dj5PhOERTTv6zZDprLIQMMaot3JYFlWi/view?usp=share_link"
)
BUDGET_PATH = get_path(
    "https://drive.google.com/file/d/1pV-iP36u1wmELu3AOLPNZ7rJJGsgj7YB/view?usp=share_link"
)
INFLATION_PATH = get_path(
    "https://drive.google.com/file/d/1k5DganllAAzXHTMLQvatpsdYy7LTlubW/view?usp=share_link"
)
PLOT_SUMMARIES_PATH = get_path(
    "https://drive.google.com/file/d/1SfJVQcRFxIaH1qfNSlu9llLK6e4TB_Bi/view?usp=share_link"
)


# local files
IMDB_MOVIES_PATH = DATA_FOLDER + "title.basics.tsv"
VIOLENT_WORDS_PATH = DATA_FOLDER + "violent_word_list.txt"

# requests
POSITIVE_WORDS_PATH = "https://ptrckprry.com/course/ssd/data/positive-words.txt"

In [ ]:
# create dataframes

# define collumn names
colnames_movies = [
    "wikipedia_movie_ID",
    "freebase_movie_ID",
    "movie_name",
    "movie_release_date",
    "movie_box_office_revenue",
    "movie_runtime",
    "movie_languages",
    "movie_countries",
    "movie_genres",
]

colnames_character = [
    "wikipedia_movie_ID",
    "freebase_movie_ID",
    "last_update",
    "character_name",
    "actor_DOB",
    "actor_gender",
    "actor_height",
    "actor_ethnicity",
    "actor_name",
    "actor_age_at_movie_release",
    "freebase_character/actor_map_ID",
    "freebase_character_ID",
    "freebase_actor_ID",
]


# load data
movies = pd.read_csv(MOVIE_PATH, sep="\t", names=colnames_movies, header=None)
characters = pd.read_csv(
    CHARACTER_PATH, sep="\t", names=colnames_character, header=None
)
budget = pd.read_csv(
    BUDGET_PATH, usecols=["id", "budget", "original_title", "release_date", "runtime"]
)

imdb_movies = pd.read_table(
    IMDB_MOVIES_PATH,
    sep="\t",
    usecols=["tconst", "originalTitle", "startYear", "runtimeMinutes"],
)
imdb_ratings = pd.read_table(RATING_PATH, sep="\t")

---
### Merging the dataframes

Movie.metadata.tsv identifies the movies with wikipedia ID, while the IMDB datasets use their own ID and the Kaggle dataset use another ID. We will use (movie name, release year, runtime) as an ID for merging the dataframes. (movie_name, release_year, runtime) makes an almost unique identifier in each of the datasets. In the cases where there are duplicates of this subset, they represent the same movie. 

In [ ]:
# adding column for release year in movies and budget dataframes.
# IMDb dataset already contains release year.
movies["year_released"] = pd.to_datetime(
    movies["movie_release_date"], infer_datetime_format=True, errors="coerce"
).dt.year

budget["year_released"] = pd.to_datetime(
    budget["release_date"], infer_datetime_format=True, errors="coerce"
).dt.year

# removing rows with null values in year_released
movies = movies[~movies["year_released"].isna()]

# renaming column names to match ('movie_name', 'release_year', 'movie_runtime') between the datasets
imdb_movies = imdb_movies.rename(
    columns={
        "startYear": "year_released",
        "originalTitle": "movie_name",
        "runtimeMinutes": "movie_runtime",
    }
)
budget = budget.rename(
    columns={"original_title": "movie_name", "runtime": "movie_runtime"}
)

# converting '\\N' values to np.nan
budget.loc[budget["year_released"] == "\\N", "year_released"] = np.nan
budget.loc[budget["movie_runtime"] == "\\N", "movie_runtime"] = np.nan

imdb_movies.loc[imdb_movies["year_released"] == "\\N", "year_released"] = np.nan
imdb_movies.loc[imdb_movies["movie_runtime"] == "\\N", "movie_runtime"] = np.nan

# removing all rows containing np.nan
budget = budget[budget["year_released"].notna()]
imdb_movies = imdb_movies[imdb_movies["year_released"].notna()]

# converting year released to 'int64'
movies["year_released"] = movies["year_released"].astype("int64")
budget["year_released"] = budget["year_released"].astype("int64", errors="ignore")
imdb_movies["year_released"] = imdb_movies["year_released"].astype("int64")

# converting runtime to 'float64'
budget["movie_runtime"] = budget["movie_runtime"].astype("float64")
imdb_movies["movie_runtime"] = imdb_movies["movie_runtime"].astype(
    "float64", errors="ignore"
)

In [ ]:
# removing rows with duplicate of ('movie_name', 'release_year', 'movie_runtime') in movies
movies = movies.drop_duplicates(subset=["movie_name", "year_released", "movie_runtime"])
budget = budget.drop_duplicates(subset=["movie_name", "year_released", "movie_runtime"])
imdb_movies = imdb_movies.drop_duplicates(
    subset=["movie_name", "year_released", "movie_runtime"]
)

In [ ]:
# merge 'movies' with 'budget'
movies = pd.merge(
    left=movies,
    right=budget,
    how="left",
    on=["movie_name", "year_released", "movie_runtime"],
)

# merge 'imdb_movies' with 'imdb_ratings':
imdb_movies = pd.merge(left=imdb_movies, right=imdb_ratings, on="tconst")

# merge 'movies' with 'imdb_movies' + 'imdb_ratings'
movies = pd.merge(
    left=movies,
    right=imdb_movies,
    how="left",
    on=["movie_name", "year_released", "movie_runtime"],
)

In [ ]:
# convert budget to float
movies["budget"] = movies["budget"].astype("float")

In [ ]:
# print the 'movie' data frame
movies.head(1)

In [ ]:
# print the 'characters' data frame
characters.head(1)

---
### Removing null values
This notebook intends to investigate the how different features effect the revenue; we will therefore filter out movies without `Movie box office revenue.`

In [ ]:
# filtering out movies without revenue
num_movies_before_filter = movies.shape[0]
movies = movies[~movies["movie_box_office_revenue"].isna()]
num_movies_after_filter = movies.shape[0]

# printing the results
print(
    f"""
Total number of movies before filtering: {num_movies_before_filter}
Number of movies after filtering:         {num_movies_after_filter}     
"""
)

#### Comment
- We observe that ~90% of the movies were filtered out.
- We still got 8 325 which should be sufficient for the data analysis.

---
# Release Date
- This section intends to investigate the runtime of the [CMU Movie Summary Corpus](http://www.cs.cmu.edu/~ark/personas/) dataset.
- The notebook will look at the distribution of release dates and how these correlate with a movie's revenue.

---
### Summary
In summary, we found a positive correlation between release year and box office revenue, and between release month and box office revenue. Because of inflation we need to be careful to interpret the increasing revenues over the year and the positive correlation. With regard to the month, we found significant differences in the mean box office revenue. The most promising month is June in terms of box office revenue. 


---
### Data Processing

In [ ]:
# column that idicate if date were given or only year
movies["date_given"] = movies["release_date"].apply(lambda x: is_given_date(str(x)))

In [ ]:
# transform the string representation to date-time representation
movies["release_date"] = pd.to_datetime(
    movies["release_date"], errors="coerce", utc=True
)

# extract year and month (if no month are given the default is set to January)
movies["release_year"] = pd.DatetimeIndex(movies["release_date"]).year
movies["release_month"] = pd.DatetimeIndex(movies["release_date"]).month

# filter out movies without release date
movies = movies[movies["release_date"].notna()]

### Plots - Year

In [ ]:
# plot the distribution of realese years
bins = np.arange(1915, 2013) - 0.5  # -0.5 for center-aligned bins
plt.hist(movies["year_released"], histtype="step", bins=bins, log=True)
plt.title("Number of movies released over the years")
plt.xlabel("year released")
plt.ylabel("number of movies")
plt.show()

In [ ]:
# plot the relation between year released and revenue
plt.scatter(movies["year_released"], movies["movie_box_office_revenue"])
plt.title("Relation between year and box office revenue")
plt.xlabel("year released")
plt.ylabel("movie box office revenue")

# removing outlier for better visualization
plt.ylim(0, 1.2e9)
plt.show()

In [ ]:
# plot the relation between year released and budget after adjusting for inflation
plt.scatter(movies["year_released"], movies["budget"])
plt.title("Relation between year and movie budget")
plt.xlabel("year released")
plt.ylabel("movie budget")
plt.show()

#### Comment

**Problem:**
- We observe that the dataset contains some old movies. The revenue and the budget will not be representable for such movies because of inflation.


**Solution:**
- A possible solution is to collect a dataset of the inflation and adjust the revenue and the budget for inflation to make them comparable.

**Conclusion:**
- We collect an external dataset on inflation. We use the Headline Consumer Price Index (CPI) of the United States to adjust for inflation from 1960 to 2014.

### Plots - Month

In [ ]:
# filter out movies without given data, these were given January as default value
month_data = movies[movies["date_given"]]

In [ ]:
# plot number of movies released over the months
plt.hist(month_data["release_month"], histtype="step", bins=10)
plt.title("Number of movies released over the months")
plt.xlabel("month")
plt.ylabel("number of movies")
plt.show()

In [ ]:
# plot mean revenue for each month
plt.plot(month_data.groupby(["release_month"])["movie_box_office_revenue"].mean())
plt.title("Mean revenue per month")
plt.xlabel("month")
plt.ylabel("revenue")
plt.show()

In [ ]:
# boxplot of the revenues for each month
sns.boxplot(x="release_month", y="movie_box_office_revenue", data=movies)
plt.title("Distributions of the boxoffice revenue during the months")
plt.xlabel("month")
plt.ylabel("box office revenue (log)")
plt.yscale("log")
plt.show()

#### Comment

- We observe that January and December has the highest frequency of releases.
- We also observe significant variation in the mean revenue for each month.

---
### Adjusting for Inflation

Our previous finding suggested that we had to adjust the `movie_box_office_revenue` for inflation to make movies more comparable.

In [ ]:
# adjusting the 'movie_box_office_revenue' for inflation
movies = correct_for_inflation(
    movies, INFLATION_PATH, "movie_box_office_revenue", start_year=1960, end_year=2014
)

# adjusting the 'budget' for inflation
movies = correct_for_inflation(
    movies, INFLATION_PATH, "budget", start_year=1960, end_year=2014
)

In [ ]:
# plot the relation between year released and revenue after adjusting for inflation
plt.scatter(movies["year_released"], movies["movie_box_office_revenue"])
plt.title("Relation between year and box office revenue")
plt.xlabel("year released")
plt.ylabel("movie box office revenue")

# removing outlier for better visualization
plt.ylim(0, 0.5e9)
plt.show()

In [ ]:
# plot the relation between year released and budget after adjusting for inflation
plt.scatter(movies["year_released"], movies["budget"])
plt.title("Relation between year and movie budget")
plt.xlabel("year released")
plt.ylabel("movie budget")
plt.show()

#### Comment
- we observe that the correlation between year and revenue is almost gone.

### Statistics
#### Correlation 
We use the Spearman Correlation Coefficient to calculate if there is a correlation between the year and the box office revenue. 

In [ ]:
# correlation between year and revenue
stats.spearmanr(movies["release_year"], movies["movie_box_office_revenue"])

- This result confirms or observation in the previous section. The correlation between year and revenue is almost gone.

#### ANOVA
In the next step we want to find if there is a best month to release a movie in terms of box office revenue. Therefore, we group the data into the different release months and check if there a significants differences between these means. To check for significant differences we perform a One-way ANOVA test.

In [ ]:
# create a list storing the revenue for each month
month_data_list = [
    month_data[month_data["release_month"] == i]["movie_box_office_revenue"]
    for i in range(1, 13)
]

# perform f_oneway
res = stats.f_oneway(*month_data_list)

# print result
print(res)

#### Comment
- Since the p-value is < 0.05 we reject the null hypothesis that the means for each month is equal.
- This result tells us that the month of realease have an impact on the movie box office revenue.

In [ ]:
# use independent t-test to check the difference between one month and the rest
results = []
months = [
    "dummy_val",
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December",
]

for i in range(1, 13):

    # divide into two populations, the month we are interested in and the rest
    month = month_data[month_data["release_month"] == i]
    all_other = month_data[month_data["release_month"] != i]

    # perform t-test
    result = stats.ttest_ind(
        month["movie_box_office_revenue"], all_other["movie_box_office_revenue"]
    )

    # perform independent t-test
    results.append({"statistic": result[0], "pvalue": result[1]})

result_df = pd.DataFrame(results, index=months[1:])
result_df

In [ ]:
# print the statistical significant months
result_df_signif = result_df[result_df["pvalue"] < 0.05]
result_df_signif

In [ ]:
# print the non-statistical significant months
result_df_non_signif = result_df[result_df["pvalue"] > 0.05]
result_df_non_signif

#### Comment
- We observe that most of the months have statistical significant different means when comparing to all the other months
- We observe that June has the lowest p-value.
- The results tells us that the month of release can have a big impact on the revenue.

---
# Diversity

- This notebook intends to investigate the diversity of the [CMU Movie Summary Corpus](http://www.cs.cmu.edu/~ark/personas/) dataset.
- The notebook will look at the distribution of ethnicities and gender and how it correlates with a movie's revenue.

---
**Summary**
- Diverse movies have, on average, ~62% higher movie box office revenue.
- Movies with the majority of male actors have, on average, ~68.8% higher movie box office revenue.
- The results are intriguing, and we would like to pursue this aspect in further analysis in milestone 3.

<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Processing" data-toc-modified-id="Data Processing">Data Processing</a></span>
<li><a href="#Distributions" data-toc-modified-id="Distributions">Distributions</a></li>
<li><a href="#Gender" data-toc-modified-id="Gender">Gender</a></li>
<li><a href="#Ethnicity" data-toc-modified-id="Ethnicity">Ethnicity</a></li>

![Diversity](./img/diversity.jpeg)

---
## Distributions

---
### Distribution of actors

In [ ]:
# add collumn for number of actors
num_actors = characters.groupby("freebase_movie_ID")["freebase_actor_ID"].apply(
    lambda x: x.count()
)

# rename the series
num_actors.name = "num_actors"

# merge with the movies data frame
movies = movies.merge(num_actors, left_on="freebase_movie_ID", right_index=True)

# print result
movies.head(1)

In [ ]:
# plot the distribution of actors
bins = np.arange(100) - 0.5  # -0.5 for center-aligned bins
plt.hist(movies["num_actors"], histtype="step", bins=bins, log=True)
plt.xlabel("Number of actors")
plt.ylabel("Number of movies")
plt.title("Distribution of actors")
plt.show()

In [ ]:
# plot relationship between 'num_actors' and 'movie_box_office_revenue'
movies.plot.scatter(x="num_actors", y="movie_box_office_revenue", s=1)
plt.yscale("log")
plt.xlabel("Number of actors")
plt.ylabel("Movie box office revenue")
plt.title("Relationship between the number actors and the movie box office revenue")
plt.show()

In [ ]:
# compute spearman correlation between 'num_actors' and 'movie_box_office_revenue'
stats.spearmanr(movies["num_actors"], movies["movie_box_office_revenue"])

#### Comment

 **Problem:**
1. We observe that many movies only have a few actors. It´s unlikely that a movie only has one actor, which is the case for ~50 movies in the dataset. The reason is probably that the dataset initially focused on characters, which can be more difficult to extract than actors.

2. We observe that movies with more actors have higher revenue. This is something to be aware of since more actors will likely provide more diversity.

**Solution:**
1. A possible solution would be to collect data about the number of actors from a different dataset. The problem with an external dataset will be finding data about ethnicity and gender. This was not included in the [IMDb datasets](https://www.imdb.com/interfaces/).

2. It will be necessary to look at the number of ethnicities and females with the total number of actors.

**Conclusion:**
1. Setting a threshold for the number of actors is a good solution that still enables us to use the original dataset.

2. Computing an `ethnicity_score` and `fraction_of_female` to also consider the total number of actors.

---
### Threshold

Our previous finding suggested that a threshold on the number of actors might be necessary.

In [ ]:
# filtering out movies with less than 10 actors
num_movies_before_filter = movies.shape[0]
movies = movies[movies["num_actors"] > 9]
num_movies_after_filter = movies.shape[0]

# printing the results
print(
    f"""
Total number of movies before filtering: {num_movies_before_filter}
Number of movies after filtering:        {num_movies_after_filter}     
"""
)

In [ ]:
# plot the distribution of actors after threshold
bins = np.arange(10, 100) - 0.5  # -0.5 for center-aligned bins
plt.hist(movies["num_actors"], histtype="step", bins=bins, log=True)
plt.xlabel("Number of actors")
plt.ylabel("Number of movies")
plt.title("Distribution of actors")
plt.show()

In [ ]:
# compute spearman correlation between 'num_actors' and 'movie_box_office_revenue'
stats.spearmanr(movies["num_actors"], movies["movie_box_office_revenue"])

#### Comment
There is still a statistically significant correlation between `num_actors` and `movie_box_office_revenue` after introducing a threshold, but the correlation is weakened.

---
## Ethnicity

---
### Correlation between the number of ethnicites and revenue

In [ ]:
# add collumn for number of ethnicities
num_ethnicities = characters.groupby("freebase_movie_ID")["actor_ethnicity"].apply(
    lambda x: x.nunique()
)

# rename the series
num_ethnicities.name = "num_ethnicities"

# merge with the movies data frame
movies = movies.merge(num_ethnicities, left_on="freebase_movie_ID", right_index=True)

In [ ]:
# plot relationship between 'num_ethnicities' and 'movie_box_office_revenue'
movies.plot.scatter(x="num_ethnicities", y="movie_box_office_revenue", s=1)
plt.yscale("log")
plt.xlabel("Number of ethnicities")
plt.ylabel("Movie box office revenue")
plt.title(
    "Relationship between the number of ethnicities and the movie box office revenue"
)
plt.show()

In [ ]:
# compute spearman correlation between 'num_ethnicities' and 'movie_box_office_revenue'
stats.spearmanr(movies["num_ethnicities"], movies["movie_box_office_revenue"])

In [ ]:
# compute spearman correlation between 'num_actors' and 'movie_box_office_revenue'
stats.spearmanr(movies["num_actors"], movies["movie_box_office_revenue"])

#### Comment
- We observe a significant correlation between `num_ethnicities` and `movie_box_office_revenue.`
- Further, we observe that the correlation between `num_actors` and `movie_box_office_revenue` is reduced after we filtered out movies with less than ten actors. This indicates that the number of actors has the most significant impact when there are few actors, e.g., increasing from 5 to 6 will have a more substantial effect on the revenue than rising from 14 to 15.
- It is still difficult to interpret how big impact `num_ethnicities` and `num_actors` have on `movie_box_office_revenue` separately since they are highly correlated.
- To try to isolate the impact ethnicities have on the revenue, we will instead look at an ethnicity_score, which is calculated as follow: $ethnicity\: score = \frac{number\,of\,ethnicities}{number\,of\,actors}$

---
### Correlation between the ethnicity score and revenue

In [ ]:
# adding a collumn for num_etnicities / num_ectors
movies["ethnicity_score"] = movies["num_ethnicities"] / movies["num_actors"]

In [ ]:
# plot relationship between 'ethnicity_score' and 'movie_box_office_revenue'
movies.plot.scatter(x="ethnicity_score", y="movie_box_office_revenue", s=1)
plt.yscale("log")
plt.xlabel("Ethnicity score")
plt.ylabel("Movie box office revenue")
plt.title("Relationship between the ethnicity score and the movie box office revenue")
plt.show()

In [ ]:
# compute spearman correlation between 'ethnicity_score' and 'movie_box_office_revenue'
stats.spearmanr(movies["ethnicity_score"], movies["movie_box_office_revenue"])

#### Comment
- After trying to isolate the impact ethnicities have on the revenue, we still observe a significant correlation.

---
### Difference in revenue between the diverse and less diverse movies

In [ ]:
# check difference between the revenue of movies with an without ethnicity score above 50
over_50_revenue = movies[movies["ethnicity_score"] > 0.5]["movie_box_office_revenue"]
under_50_revenue = movies[movies["ethnicity_score"] <= 0.5]["movie_box_office_revenue"]

In [ ]:
# independent t-test
stats.ttest_ind(over_50_revenue, under_50_revenue)

In [ ]:
# calculate the statistical power

# creating list to store the p-values
p_val = np.zeros(10000)

# running the simulation 10000 times
for i in range(10000):

    # sample over_50 and under_50
    sample_over_50 = np.random.choice(over_50_revenue, size=over_50_revenue.shape[0])
    sample_under_50 = np.random.choice(under_50_revenue, size=under_50_revenue.shape[0])

    # independent t-test
    p_val[i] = stats.ttest_ind(sample_over_50, sample_under_50)[1]

# calculating the percentage of rejects
power = len(p_val[p_val < 0.05]) / len(p_val)

# printing the results
print(
    f"""
The statistical power of:
    - The independent sample t-test comparing the revenue of diveres and less diverse movies: {power}
"""
)

In [ ]:
# calculating the mean difference
mean_diff_diversity = over_50_revenue.mean() - under_50_revenue.mean()

# performing bootstrap to get the 95% CI
diversity_CI = bootstrap_CI(over_50_revenue, under_50_revenue)

# printing the results
print(
    f"""
The mean difference between in revenue between diverse and less diverse movies:
    - Mean:                       {mean_diff_diversity}
    - CI:                         {diversity_CI}
"""
)

In [ ]:
# calculating the average revenue for under_50
mean_under_50 = under_50_revenue.mean()

# calculating the percentage difference in revenue
diff_percent = mean_diff_diversity / mean_under_50

# printing the results
diff_percent

#### Comment 
- The independent t-test gives us a small p-value (< 0.05), which indicates a statistically significant difference between the samples of diverse (ethnicity score > 0.5) and less diverse (ethnicity score ≤ 50) movies. The statistical power of the result is 0.8765. 
- Further, the mean difference shows that diverse movies generally have higher movie box office revenue. The confidence interval makes us confident in this result. 
- By dividing the average difference by the average revenue for less diverse movies, we get that diverse movie, on average, have a ~62% higher movie box office revenue. This shows that ethnicity is impactful on the movie box office revenue.

---
## Gender

---
### Correlation between the number of females and revenue

In [ ]:
# checkint the amount of characters without 'actor_gender'
num_characters = characters.shape[0]
num_nan_gender = characters[characters["actor_gender"].isna()].shape[0]

# printing the results
print(
    f"""
Total number of characters:                {num_characters}
Number of characters without actor gender:  {num_nan_gender}     
"""
)

In [ ]:
# add collumn for fraction of females

# create frame with female actors
female_characters = characters[characters["actor_gender"] == "F"]

# add column for number of female actors
num_female = female_characters.groupby("freebase_movie_ID")["actor_gender"].count()

# rename the series
num_female.name = "num_female"
movies = movies.merge(num_female, left_on="freebase_movie_ID", right_index=True)

# print result
movies.head(1)

In [ ]:
# plot relationship between 'num_female' and 'movie_box_office_revenue'
movies.plot.scatter(x="num_female", y="movie_box_office_revenue", s=1)
plt.yscale("log")
plt.xlabel("Number of females")
plt.ylabel("Movie box office revenue")
plt.title("Relationship between number of females and movie box office revenue")
plt.show()

In [ ]:
# compute spearman correlation between 'num_female' and 'movie_box_office_revenue'
stats.spearmanr(movies["num_female"], movies["movie_box_office_revenue"])

#### Comment 
- We observe a weak but significant correlation between `num_female` and `movie_box_office_revenue.`
- It is difficult to interpret how big an impact `num_female` and `num_actors` have on `movie_box_office_revenue` separately since they are highly correlated (more actors will likely lead to more female actors as well).
- To try to isolate the impact gender have on the revenue, we will instead look at the fraction of female, which is calculated as follow: $fraction\: of\: female = \frac{number\,of\,female}{number\,of\,actors}$

---
### Correlation between the fraction of females and revenue

In [ ]:
# add column for fraction of females
movies["frac_female"] = movies["num_female"] / movies["num_actors"]

# print result
movies.head(1)

In [ ]:
# plot relationship between 'frac_female' and 'movie_box_office_revenue'
movies.plot.scatter(x="frac_female", y="movie_box_office_revenue", s=1)
plt.yscale("log")
plt.xlabel("Fraction of females")
plt.ylabel("Movie box office revenue")
plt.title("Relationship between fraction of females and movie box office revenue")
plt.show()

In [ ]:
# compute spearman correlation between 'frac_female' and 'movie_box_office_revenue'
stats.spearmanr(movies["frac_female"], movies["movie_box_office_revenue"])

#### Comment
- After trying to isolate the impact on the revenue, we still observe a significant correlation, but this time we observe a negative correlation. The correlation between `frac_females` and `movie_box_office_revenue` is weak.

---
### Difference in revenue between the movies with majority of male and female actors

In [ ]:
# check difference between the revenue of movies with majority of men and majority of women
majority_male_revenue = movies[movies["frac_female"] <= 0.5]["movie_box_office_revenue"]
majority_female_revenue = movies[movies["frac_female"] > 0.5][
    "movie_box_office_revenue"
]

In [ ]:
# independent t-test
stats.ttest_ind(majority_male_revenue, majority_female_revenue)

In [ ]:
# calculate the statistical power

# creating list to store the p-values
p_val = np.zeros(10000)

# running the simulation 10000 times
for i in range(10000):

    # sample over_50 and under_50
    sample_majority_male = np.random.choice(
        majority_male_revenue, size=majority_male_revenue.shape[0]
    )
    sample_majority_female = np.random.choice(
        majority_female_revenue, size=majority_female_revenue.shape[0]
    )

    # independent t-test
    p_val[i] = stats.ttest_ind(sample_majority_male, sample_majority_female)[1]

# calculating the percentage of rejects
power = len(p_val[p_val < 0.05]) / len(p_val)

# printing the results
print(
    f"""
The statistical power of:
    - The independent sample t-test comparing the revenue of movies with majority of male and majority of female 
      actors: {power}
"""
)

In [ ]:
# calculating the mean difference
mean_diff_gender = majority_male_revenue.mean() - majority_female_revenue.mean()

# performing bootstrap to get the 95% CI
gender_CI = bootstrap_CI(majority_male_revenue, majority_female_revenue)

# printing the results
print(
    f"""
The mean difference between in revenue between diverse and less diverse movies:
    - Mean:                       {mean_diff_gender}
    - CI:                         {gender_CI}
"""
)

In [ ]:
# calculating the average revenue for under_50
mean_majority_female = majority_female_revenue.mean()

# calculating the percentage difference in revenue
diff_percent = mean_diff_gender / mean_majority_female

# printing the results
diff_percent

#### Comment
- The independent t-test gives us a small p-value (<< 0.05), which indicates a statistically significant difference between the samples with the majority of male actors (fraction of females ≤ 0.5) and the samples with the majority of female actors (fraction of females > 50). The statistical power of 0.9973 makes us confident in this result.
- Further, the mean difference tells us that the movies with the majority of male actors generally have a higher movie box office revenue. The confidence interval makes us confident in this result. 
- By dividing the average difference by the average revenue for movies with the majority of female actors, we get that movies with the majority of male actors, on average, have a ~68.8% higher movie box office revenue. This shows that the gender-composition is impactful on the movie box office revenue.

# Cast

- This section intends to investigate the casts of the CMU Movie Summary Corpus dataset.
- The notebook will do some initial analyses to see how individual movie actors affects the box office revenue. The aim of the analysis is to get an idea of relevant methods we could use, and understand how we can handle the data and data size. 

---
**Summary**

- If we include all actors in a linear regression model, we exceed the maximum recursion depth running the model. In addition to this, we get multicollinearity problems if we do not restrict the amount of actors in our regression model. 
- By using all actors that have played in more than 10 movies as categorical predictors, our model explains 59% of the variance in box office revenue, with an adjusted R-squared equal to 50%. Using actors that have played in more than 15 movies as predictors explains 32% of the variance in box office revenue, with an adjusted R-squared equal to 27%.
- Among the actors (as predictors) with p-value < 0.05, Alan Rickman (Professor Severus Snape) received the highest coefficient in our linear regression model. PS: This is only initial analysis and we will explain why the result cannot be trusted yet. Spoiler alert: Confounders.

---
## Preprocessing

---
### Loading data

In [ ]:
# load data
movies = pd.read_csv(MOVIE_PATH, sep="\t", names=colnames_movies, header=None)
characters = pd.read_csv(
    CHARACTER_PATH, sep="\t", names=colnames_character, header=None
)

---
### Null Values

In [ ]:
movies.isna().sum()

In [ ]:
characters.isna().sum()

#### Comment
Both box office revenue and freebase actor ID contain some Null values we want to remove before exploring the cast's effect on revenue made. 

In [ ]:
# Removing characters/actors without freebase_actor_id
characters = characters[characters["freebase_actor_ID"].notna()]

# Removing movies without box_office_revenue
movies = movies[movies["movie_box_office_revenue"].notna()]

---
### Correcting for inflation

We want to compare movies of different years from 2000 to 2012, so we need to correct for inflation.

In [ ]:
# adjusting the 'movie_box_office_revenue' for inflation
movies = correct_for_inflation(
    movies, INFLATION_PATH, "movie_box_office_revenue", start_year=2000, end_year=2012
)

---
### Filtering and merging
We will analyze our data to create a recipe for a successful movie. For this section, we want to explore if certain actors impact the success of a movie. We want to restrict the time interval to movies released after 2000. Then we get a higher fraction of actors still in the movie industry.

In [ ]:
# Removing movies from before 2000
movies = movies[movies["movie_release_date"] >= "2000"]

In [ ]:
# Merging movies with characters on wiki_id.
movies_characters = pd.merge(
    left=movies, right=characters, on=["wikipedia_movie_ID", "freebase_movie_ID"]
)

In [ ]:
movie_count = movies_characters.shape[0]
unique_combos = movies_characters.value_counts(
    subset=["wikipedia_movie_ID", "freebase_actor_ID"], dropna=False
).shape[0]

print(
    """
    Total number of movies (with duplicates) in our merged dataset: {}
    Unique number of combinations of 'wikipedia_movie_ID' and 'freebase_actor_ID': {}
    """.format(
        movie_count, unique_combos
    )
)

#### Comment
Some actors are listed mulitple times because they play different characters in the same movie. We only want unique combinations of `wikipedia_movie_ID` and `freebase_actor_ID`. 


In [ ]:
# We filter out duplicated combinations of 'wikipedia_movie_ID' and 'freebase_actor_ID'.
# For now, we do not care which row/character we keep
movies_characters = movies_characters.drop_duplicates(
    subset=["wikipedia_movie_ID", "freebase_actor_ID"], keep="first"
)
movies_characters.shape

---
## Understanding Cast Data

In [ ]:
# Number of unique movies
num_movies = movies_characters["wikipedia_movie_ID"].nunique()
print("Number of unique movies: {}".format(num_movies))

In [ ]:
# Number of unique actors in dataset
num_actors_unique = movies_characters["freebase_actor_ID"].nunique()
print("Number of unique actors: {}".format(num_actors_unique))

In [ ]:
# Checking number of actors in dataset, counting actors multiple times if they play
# in multiple movies
count_actors = movies_characters["freebase_actor_ID"].count()
print("Number of rows containing freebase actor ID: {}".format(count_actors))

In [ ]:
# Number of actors on average per movie
ratio = count_actors / num_movies
print("Number of actors on average per movie: {}".format(ratio))

---
## Exploring method: Linear Regression

---
### One Hot Encoding of Actors

In [ ]:
# One Hot Encoding of Actors
movies_characters_dummy = pd.get_dummies(
    data=movies_characters, columns=["freebase_actor_ID"]
)
movies_characters_dummy.shape

In [ ]:
# Only include wikipedia movie ID and one hot encoding of actors in dataframe
dummy_actor_columns = movies_characters_dummy.filter(
    regex="wikipedia_movie_ID|freebase_actor_ID_"
)

In [ ]:
# We want to try different thresholds for how many actors we include.
# Exploring actors that have played in more than 10 and 15 movies.
dummy_actor_columns15 = dummy_actor_columns.loc[:, dummy_actor_columns.sum(axis=0) > 15]
dummy_actor_columns10 = dummy_actor_columns.loc[:, dummy_actor_columns.sum(axis=0) > 10]
print(
    "Number of actors that have played in more than 10 movies: {}".format(
        dummy_actor_columns10.shape[1] - 1
    )
)
print(
    "Number of actors that have played in more than 15 movies: {}".format(
        dummy_actor_columns15.shape[1] - 1
    )
)

#### Comment
By only including actors that have played in more than 10 movies, we reduce the dataframe size from 22953 to 647. Reducing the threshold further to 15, we reduce the dataframe size from 22953 columns to 247 columns. This makes the linear regression possible without exceeding the maximum recursion depth. There might be actors excluded who actually have a big impact on revenue made, but for now, we want to explore the actors that have played in more than 15 movies. 

In [ ]:
# Grouping movies such that every movie correspond to only one row in the dataframe
dummy_actor_columns10 = dummy_actor_columns10.groupby("wikipedia_movie_ID").agg("sum")
dummy_actor_columns15 = dummy_actor_columns15.groupby("wikipedia_movie_ID").agg("sum")

# Merging dummy actor columns with the movies
movies_binary_actors10 = pd.merge(
    movies, dummy_actor_columns10, left_on="wikipedia_movie_ID", right_index=True
)
movies_binary_actors15 = pd.merge(
    movies, dummy_actor_columns15, left_on="wikipedia_movie_ID", right_index=True
)
movies_binary_actors10.head(1)

---
### Linear Regression

In [ ]:
# Removing slashes in column names to avoid error in regression.
movies_binary_actors10.columns = movies_binary_actors10.columns.str.replace("/", "")
dummy_actor_columns10.columns = dummy_actor_columns10.columns.str.replace("/", "")

movies_binary_actors15.columns = movies_binary_actors15.columns.str.replace("/", "")
dummy_actor_columns15.columns = dummy_actor_columns15.columns.str.replace("/", "")

In [ ]:
# Constructing formula used for regression
# For now we only include actors as categorical predictors
# We have two scenarios:
# 1) All actors that have played in more than 10 movies
# 2) All actors that have played in more than 15 movies
formula10 = "movie_box_office_revenue ~ "
for col in dummy_actor_columns10.columns:
    formula10 += "C(" + col + ")+"

formula15 = "movie_box_office_revenue ~ "
for col in dummy_actor_columns15.columns:
    formula15 += "C(" + col + ")+"

formula10 = formula10[:-1]
formula15 = formula15[:-1]

In [ ]:
# Linear regression
mod10 = smf.ols(formula=formula10, data=movies_binary_actors10)
res10 = mod10.fit()
res_summary10 = res10.summary()

mod15 = smf.ols(formula=formula15, data=movies_binary_actors15)
res15 = mod15.fit()
res_summary15 = res15.summary()

In [ ]:
print(
    "Our model with actors that played in more than 10 movies gets R-squared = {:.2f}".format(
        res10.rsquared
    )
)
print("The corresponding adjusted R-squared is: {:.2f}\n".format(res10.rsquared_adj))

print(
    "Our model with actors that played in more than 15 movies gets R-squared = {:.2f}".format(
        res15.rsquared
    )
)
print("The corresponding adjusted R-squared is: {:.2f}".format(res15.rsquared_adj))

#### Comment
Our model with actors that have played in more than 10 movies explains 59% of the variance in  box office revenue. However, we have many variables included (actors), which could make our model look more accurate even if there are many actors poorly contributing in some way. The adjusted R-squared score (50%) may be telling us that some variables (actors) are not contributing to our model’s R-squared properly. 

When we only include actors that have played in more than 15 movies, we get a lower R-squared, but there will also be less multicollinearity problems. 

For now we have only used actors as predictors, so there are likely to be confounders. Confounders could be budget, genre, release_date and more. However, the large R-squared tells us that we should continue exploring how actors and cast affect the box office revenue. 

In [ ]:
# Turning result summary into a dataframe
# (actors that have played in more than 15 movies)
res_as_html = res_summary15.tables[1].as_html()
summary_df = pd.read_html(res_as_html, header=0, index_col=0)[0]

In [ ]:
# Only including actors with p-value < 0.05.
# Statistically significant actors
summary_df = summary_df[summary_df["P>|t|"] < 0.05]

# Sorting the dataframe in descending order according to coefficient
summary_df.sort_values(by="coef", ascending=False, inplace=True)

# The top 3 actors with highest coefficient
summary_df.head(3)

In [ ]:
# Checking the number 1 actor we received from our
# initial analysis (by only considering actors as predictor)

movies_characters[movies_characters["freebase_actor_ID"] == "/m/09y20"].sort_values(
    by="movie_box_office_revenue", ascending=False
).head(3)

#### Comment
We cannot be sure whether Alan Rickman aka Professor Severus Snape has the highest effect on box office revenue. However, Alan Rickman has performed  in several movies with extremely high box office revenue, so the result is an indication that our choice of method could be further explored to gain insight into what actors affect box office revenue the most. The most urgent improvement of the model is to include more predictors (both categorical and continous) to avoid confounders.








![snape](img/snape.jpeg)

# Runtime

- This section intends to investigate the runtime of the [CMU Movie Summary Corpus](http://www.cs.cmu.edu/~ark/personas/) dataset.
- The notebook will look at the distribution of runtime and how it correlates with a movie's revenue.

---
### Summary
- We cannot conclude there is a linear relationship between movie runtime and revenue.
- Blockbuster movies ($400 million+) have, on average, 10.1% longer runtime than non-blockbuster movies.
- Movies with short runtime (<80min) have 54.8% less revenue than non-short movies.

---
### Data Processing

---
#### Loading data

In [ ]:
# Filter for movies with runtime
df_runtime = movies[movies["movie_runtime"].notnull()]

---
#### Outlier Removal
We will find outliers by calculating IQR and remove data outside the lower/upper bound.

In [ ]:
# IQR
Q1 = np.percentile(df_runtime["movie_runtime"], 25)
Q3 = np.percentile(df_runtime["movie_runtime"], 75)
IQR = Q3 - Q1
lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR

# Remove outlier
df_runtime = df_runtime[
    (df_runtime["movie_runtime"] >= lower) & (df_runtime["movie_runtime"] <= upper)
]
df_runtime.head(1)

---
### Distribution of runtime

In [ ]:
# histogram plot
plt.hist(df_runtime["movie_runtime"], histtype="step", log=True, bins=20)
plt.title("Distribution of movie runtime")
plt.xlabel("Length of movie runtime (minute)")
plt.ylabel("Number of movies")
plt.show()

#### Comment

From the log histogram, we can see that most of the movies have runtime within the 85-120 minute interval.

---
### Regression & Scatterplot

In [ ]:
runtime_model = smf.ols(
    "movie_box_office_revenue ~ movie_runtime", data=df_runtime
).fit()

# summarize our model
runtime_model_summary = runtime_model.summary()
runtime_model_summary

In [ ]:
fig = plt.figure(figsize=(9, 6))

# pass in the model as the first parameter, then specify the
# predictor variable we want to analyze
fig = sns.regplot(
    x="movie_runtime",
    y="movie_box_office_revenue",
    data=df_runtime,
    ci=95,
    scatter_kws={"color": "blue"},
    line_kws={"color": "red"},
)

#### Comment
- From the scatterplot, we see that there is a slight positive linear relationship between runtime and revenue. 
- However from previous regression analysis, the R squared value (0.035) is too small to justify the dependency between runtime and revenue. 
- We also see that movie with runtime between 80-120 minutes have similar distribution of revenue and movies with runtime less than 80 minutes generate less revenue (highet revenue is below $400000000). - Thus we can raise the question that if 80 minute could be a threshold for movie's revenue performance and we will address it in the next section.

---
### Effect of short runtime on revenue
We define **short** movie to have runtime < 80min\
and other (non-short) movie to have runtime >= 80min.

In [ ]:
# Filter short movie to be under 80min
df_short = df_runtime[df_runtime["movie_runtime"] < 80]

# Filter non-short movie to be above or equal to 80min
df_non_short = df_runtime[df_runtime["movie_runtime"] >= 80]

In [ ]:
# independent t-test between short and non-short movies
stats.ttest_ind(
    a=df_short["movie_box_office_revenue"],
    b=df_non_short["movie_box_office_revenue"],
    equal_var=False,
)

In [ ]:
# calculate mean difference
mean_diff = (
    df_short["movie_box_office_revenue"].mean()
    - df_non_short["movie_box_office_revenue"].mean()
)

# print the mean difference
print(f"The mean difference in revenue is: {mean_diff}")

In [ ]:
# calculating the percentage difference in revenue between the mean and non-short
diff_percent = mean_diff / df_non_short["movie_box_office_revenue"].mean()

# printing the results
diff_percent

#### Comment
- The independent t-test gives us a small p-value (<< 0.05), which indicates a statistically significant difference between the samples.
- Further, the mean difference tells us that short movies (< 80min) generally generates less revenue than non-short (>=80min) movies.
- We can conclude that movies should not be too short as short movies have 54.8% less revenue than non-short movies.

---
### Difference in runtime between blockbuster vs. non-blockbuster
We define **blockbusters** as movies with revenue >= 400 million USD\
and **non-blockbusters** as movies with revenue < 400 million USD.

In [ ]:
# Filter blockbuster movies
df_bb = df_runtime[df_runtime["movie_box_office_revenue"] >= 400000000]

# Filter non-blockbuster movies
df_non_bb = df_runtime[df_runtime["movie_box_office_revenue"] < 400000000]

In [ ]:
# independent t-test
stats.ttest_ind(a=df_bb["movie_runtime"], b=df_non_bb["movie_runtime"], equal_var=False)

In [ ]:
# calculate mean difference
mean_diff = df_bb["movie_runtime"].mean() - df_non_bb["movie_runtime"].mean()

# print the mean difference
print(f"The mean difference in runtime is: {mean_diff}")

In [ ]:
# calculating the percentage difference in runtime between the mean and non-blockbuster
diff_percent = mean_diff / df_non_bb["movie_runtime"].mean()

# printing the results
diff_percent

#### Comment
- The independent t-test gives us a small p-value (<< 0.05), which indicates a statistically significant difference between the samples.  
- Further, the mean difference tells us that the blockbuster runtime is generally longer than non-blockbuster.  
- We can conclude that longer runtime are more successful, as blockbuster movies have 10.05% longer runtime than non-blockbuster movies. 

# Plot Summary

- This section intends to investigate the plot summary of the [CMU Movie Summary Corpus](http://www.cs.cmu.edu/~ark/personas/) dataset.
- The notebook will look at the sentiments within plot summary of movies and how they correlate with a movie's revenue.

---
### Summary
- We analyze plot summary from three sentiments: positive, negative and violent words and analyze if their proportion in plot summary affects revenue.
- Movies with more negative plot have on average 45.3% more revenue than those with more positive plots.
- Movies with more violent plot are more likely to generate revenue since blockbuster movies have 36.9% higher violent word porportion than non-blockbuster movies.

---
### Data Processing

---
#### Loading data

In [ ]:
# Read summary data and filter movies with summary not null
df_summary = pd.read_csv(
    PLOT_SUMMARIES_PATH, sep="\t", header=None, names=["wikipedia_movie_ID", "summary"]
)
df_summary = df_summary[df_summary["summary"].notnull()]

In [ ]:
# Merge metadata and summary data and filter for revenue and summary
df_merge = pd.merge(movies, df_summary, on=["wikipedia_movie_ID"], how="inner")
df_merge = df_merge[["wikipedia_movie_ID", "summary", "movie_box_office_revenue"]]
df_merge.head(5)

---
#### Positive / Negative / Violent Vocab Collection
We decide to examine if positivity, negativity and violence in plot will affect movie revenue.\
First, we collect vocabulary with positive, negative and violent connotation.

In [ ]:
# Sentiment List
sentiment = ["positive", "negative", "violent"]

In [ ]:
# collecting the positive words list

# requesting the url
r = requests.get(POSITIVE_WORDS_PATH)

# extracting and decoding the content from the request
text = r.content.decode("us-ascii")

# extracting the text containing the positive words
pos_text = text.split("\n\n")

# converting the text into a list
pos_list = pos_text[1].split("\n")

In [ ]:
# collecting the neagative words list

# requesting the url
r = requests.get("https://ptrckprry.com/course/ssd/data/negative-words.txt")

# extracting and decoding the content from the request
text = r.content.decode("iso-8859-1")

# extracting the text containing the negative words
neg_text = text.split("\n\n")

# converting the text into a list
neg_list = neg_text[1].split("\n")

In [ ]:
# collecting the violence words list
# source: https://myvocabulary.com/word-list/violence-vocabulary/

# Read from txt file
violent_file = open(VIOLENT_WORDS_PATH, "r")
violent_words = violent_file.read()

# Split into list
vio_list = violent_words.split(", ")

# Map every word into lower case
vio_list = list(map(lambda x: x.lower(), vio_list))

---
#### Positive / Negative / Violent proportion in summary
We calculate the proportion of words with positive / negative / violent connotation out of all words in movie summary.

In [ ]:
# adding column for positive sentiment
df_merge["positive"] = df_merge["summary"].apply(lambda x: check_summary(x, pos_list))

# adding a columns for negative sentiment
df_merge["negative"] = df_merge["summary"].apply(lambda x: check_summary(x, neg_list))

# adding a columns for violent sentiment
df_merge["violent"] = df_merge["summary"].apply(lambda x: check_summary(x, vio_list))

In [ ]:
df_merge.head(2)

---
### Regression & Scatterplot

In [ ]:
fig, ax = plt.subplots(3, 1)

# sentiment to ax position mapping
sen2ax = {"positive": ax[0], "negative": ax[1], "violent": ax[2]}

# Plot for each sentiment: positive, negative, violent
for s in sentiment:
    ax = sen2ax[s]
    ax.plot(df_merge[s], df_merge["movie_box_office_revenue"], "o")

    # obtain m (slope) and b(intercept) of linear regression line
    m, b = np.polyfit(df_merge[s], df_merge["movie_box_office_revenue"], 1)
    # add linear regression line to scatterplot
    ax.plot(df_merge[s], m * df_merge[s] + b)

    ax.set_title(f"Number of {s} words vs movie_box_office_revenue")
    ax.set_xlabel(f"{s} words count")
    ax.set_ylabel("Revenue")
    ax.set_ylim(0, 3000000000)

# Combine all the operations and display
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=1.7, wspace=0.4, hspace=0.4)
plt.show()

In [ ]:
# Regression analysis for positive word proportion vs revenue
positive_revenue_model = smf.ols(
    "movie_box_office_revenue ~ positive", data=df_merge
).fit()
positive_revenue_model_summary = positive_revenue_model.summary()
positive_revenue_model_summary

In [ ]:
# Regression analysis for negative word proportion vs revenue
negative_revenue_model = smf.ols(
    "movie_box_office_revenue ~ negative", data=df_merge
).fit()
negative_revenue_model_summary = negative_revenue_model.summary()
negative_revenue_model_summary

In [ ]:
# Regression analysis for violent word proportion vs revenue
violence_revenue_model = smf.ols(
    "movie_box_office_revenue ~ violent", data=df_merge
).fit()
violence_revenue_model_summary = violence_revenue_model.summary()
violence_revenue_model_summary

#### Comment
- From the scatterplot, we see that there is a negative linear relationship between positive word proportion and revenue. In contrast, there is a positive linear relationship between negative word proportion and revenue. There is also a positive linear relationship between violent word proportion and revenue. 
- However from previous regression analysis, the R squared value (0.004 - positive, 0.000 - negative, 0.008 - violent) is too small to justify the dependency between all sentiments and revenue. 
- We see a opposing relationship between positive and negative plots and we will discover if more positive plot or more negative plot can boost the revenue in the following section.

---
### Difference in revenues between movies with more positive and more negative plot
We will analyze if plots with more positive word proportions will generate more revenues than those with more negative word proportions. 

In [ ]:
# Filter for movies that have higher proportion of positive words than negative words
df_pos_majority = df_merge[df_merge["positive"] > df_merge["negative"]]

# Filter for movies that higher proportion of negative words than positive words
df_neg_majority = df_merge[df_merge["negative"] > df_merge["positive"]]

In [ ]:
# independent t-test
stats.ttest_ind(
    a=df_pos_majority["movie_box_office_revenue"],
    b=df_neg_majority["movie_box_office_revenue"],
    equal_var=False,
)

In [ ]:
# calculate mean difference
mean_diff = (
    df_pos_majority["movie_box_office_revenue"].mean()
    - df_neg_majority["movie_box_office_revenue"].mean()
)

# print the mean difference
print(f"The mean difference in revenue is: {mean_diff}")

In [ ]:
# calculating the percentage difference in revenue between the mean and negative
diff_percent = mean_diff / df_neg_majority["movie_box_office_revenue"].mean()

# printing the results
diff_percent

#### Comment
- The independent t-test gives us a small p-value (<< 0.05), which indicates a statistically significant difference between the samples.
- Further, the mean difference tells us that movies with more positive plot generally has less revenue than movies with more negative plot.
- We can conclude that movies with more negative plot are more successful, as movies with more negative plot have 45.3% more revenue than movies with more positive plot.

---
### Difference in sentiment proportion between blockbuster vs. non-blockbuster
We define **blockbusters** as movies with revenue >= 400 million USD\
and **non-blockbusters** as movies with revenue < 400 million USD.

In [ ]:
# Filter blockbuster movies
df_bb = df_merge[df_merge["movie_box_office_revenue"] >= 400000000]

# Filter non-blockbuster movies
df_non_bb = df_merge[df_merge["movie_box_office_revenue"] < 400000000]

In [ ]:
# For each sentiment
for s in sentiment:
    # independent t-test
    print(stats.ttest_ind(a=df_bb[s], b=df_non_bb[s], equal_var=False))

    # calculate mean difference
    mean_diff = df_bb[s].mean() - df_non_bb[s].mean()

    # print the mean difference
    print(f"The mean difference in proportion of {s} words in summary is: {mean_diff}")

    # calculating the percentage difference between the mean and non-blockbuster
    diff_percent = mean_diff / df_non_bb[s].mean()

    # printing the results
    print(
        f"The percentage mean difference in proportion of {s} words in summary between blockbuster and non-blockbuster is: {diff_percent * 100}%\n"
    )

#### Positive
- The independent t-test for positive words gives us a small p-value (<< 0.05), which indicates a statistically significant difference between the samples.  
- Further, the mean difference tells us that blockbuster movies generally has smaller positive word proportion in summary than non-blockbuster.  
- We can conclude that less positive plot components are more successful, as blockbuster movies have 10.2% less positive word usage than non-blockbuster movies. 

#### Negative
- The independent t-test for negative words gives us a small p-value (<< 0.05), which indicates a statistically significant difference between the samples.  
- Further, the mean difference tells us that blockbuster movies generally has higher negative word proportion in summary than non-blockbuster.  
- We can conclude that more negative plot components are more successful, as blockbuster movies have 12.4% more negative word proportion than non-blockbuster movies. 

#### Violent
- The independent t-test for violent words gives us a small p-value (<< 0.05), which indicates a statistically significant difference between the samples.  
- Further, the mean difference tells us that blockbuster movies generally has higher violent word proportion in summary than non-blockbuster.  
- We can conclude that more violent plot components are more successful as blockbuster movies have 36.9% higher violent word proportion than non-blockbuster movies. 